# web scraping RIP.ie with beautifulsoup

In [37]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [2]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [40]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

238144

## iterate over a range of ids and get info

In [ ]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(450000,462029):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)    
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

In [145]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

In [146]:
res

,id,name,date,county,address,notice
0,461783,William Barrow,2021-06-19,Cork,"Canon Corbett Place, Mallow, Cork","The death has occurred of William Barrow, late..."
1,461784,Michael Wall,2021-06-20,Tipperary,"Carrick-on-Suir, Tipperary","Michael Wall, late of 6 Castle View, Carrick o..."
2,461785,Jane Lucey,2021-06-20,Cork,"Kielnamoul, Bandon, Cork","Lucey (Kielnamoul, Bandon) on June 20th 2021 i..."
3,461786,Sarah (Sally) Quirke (née Feeney),2021-06-19,Galway,"Fursey Road, Shantalla, Galway / Inverin, Galway",Formerly Inverin Galway.;Sarah (Sally) died pe...
4,461787,Mary Ring (née Delaney),2021-06-19,Wexford,"Rocklane, Adamstown, Wexford / Enniscorthy, W...",Peacefully at home in the loving care of her f...
...,...,...,...,...,...,...
241,462024,Gerry KILDUFF,2021-06-19,Dublin,"Glenageary, Dublin","KILDUFF Gerry (Glenageary, Co. Dublin) June 19..."
242,462025,Maureen McGirr (née Fitzpatrick),2021-06-22,Meath,"Kilbarrack Avenue, Raheny, Dublin","Maureen McGirr (nee Fitzpatrick), Kilbarrack A..."
243,462026,Margaret Egan (née Wall),2021-06-22,Offaly,"Borris-in-Ossory, Laois / Ballaghmore, Laois ...","Margaret Egan nee Wall, Kyle, Ballaghmore, Bor..."
244,462027,Bridie Donohoe (née Gill),2021-06-22,Leitrim,"Ardunsaghan, Carrigallen, Leitrim / Killeshan...",Unexpectedly and peacefully at her residence.P...


In [147]:
x = pd.concat([df,res]).reset_index(drop=True)
x=x[~x.id.duplicated(keep='first')]
print (len(df),len(res),len(x))
x.to_pickle('rip_dn_scrape.pkl')

461783 246 462029


In [148]:
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
#x=x.drop_duplicates(['name','date','county'])
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

462029
443385


## test matching

In [ ]:
targets=pd.DataFrame([('Lottie Graham','BigPark, Ballintra, Co Donegal'),('a','b')],columns=('name','address'))

In [ ]:
targets

def find_match(x,s):
    if s in str(x.notice):
        return True
    return False

s='Rialto'
x=res.apply(lambda x: find_match(x,s),1)
res[x]

In [48]:
dup=x[x.duplicated(['name','address'])]

In [38]:
x.loc[390559]

name                                            David  Moran
date                                              08/06/2019
address                 Cedar Court, Kennedy Park,  Limerick
notice     Late of John Carew Park.;David, passed away at...
county                                              Limerick
Name: 390559, dtype: object